In [61]:
import pygame
import random
import math

sound1 = 'ding.mp3'
imgelv = "elv.png"
imgfloor = "b.png"
imgpassenger = "men1.png"
a4=1/10
a9=0
a88=1
class building:

    def __init__(self, screen, floorheight, floorwidth, numofelv=2, space=1,numfloors=17):
        self.numfloors=numfloors
        self.screen = screen
        self.elevators = [elevator(screen, floorwidth//2 + j * 80, floorheight, (numfloors - 1) * floorheight) for j in range(numofelv)]


        self.numfloors=numfloors
        self.floors = [floor(screen, 0, (self.numfloors - 1 - i) * floorheight, floorheight, floorwidth//2, i, self) for i in range(self.numfloors)]
        
    def update(self, floor):
        selectedelevator = self.elevators[0]
        mindistance = self.numfloors*20
        for elevator in self.elevators:
            
            if elevator.ismovingindirection(floor.floornum, elevator.direction):
                distance = abs(elevator.myfloor - floor.floornum)
                
                if distance < mindistance:
                
                    mindistance = distance
                    selectedelevator = elevator
                
        selectedelevator.addtarget(floor)
        
        #floor.timewait=mindistance*0.5
        selectedelevator.moving=True
    def update1(self,dt):
        for i in self.elevators:
            
            if i.moving==True:
              i.move(dt)
            i.draw()

    def draw(self):
        for floor in self.floors:
            floor.draw()
        for elevator in self.elevators:
            elevator.draw()

class elevator:
    def __init__(self, screen, initialposx, floorheight, initial_ypos):
        self.screen = screen
        self.ypos = initial_ypos
        self.xpos = initialposx
        self.floorheight = floorheight
        self.step = floorheight / 100
        self.myfloor = 0
        self.direction = None
        self.targets = []
        self.passengers = []
        self .pusze=0
        pygame.mixer.init()
        self.dingsound = pygame.mixer.Sound(sound1)
        self.moving = False
        self.time1=0
        self.time2=0
        self.time3=pygame.time.get_ticks()
        originalimage = pygame.image.load(imgelv).convert_alpha()
        self.image = pygame.transform.scale(originalimage, (75, int(floorheight * 0.75)))
        self.rect = self.image.get_rect(topleft=(self.xpos, self.ypos))
        
    def addtarget(self, floor):
        
        if floor not in self.targets:
            self.targets.append(floor)
            self.targets.sort(key=lambda x: x.floornum, reverse=self.direction == 1)
        if self.direction is None:
            self.setdirection(floor.floornum)
        floor.timewait=self.calculate_wait_time(self.targets,self.myfloor,floor.floornum,self.direction)
    
    def calculate_wait_time(self,targets, current_floor, new_floor, direction):
        
        initial_distance = abs(current_floor - new_floor)
        initial_wait_time = initial_distance * 0.5
        index = 0
        if direction == 1:
            while index < len(targets) and targets[index].floornum > new_floor:
                index += 1
        else:
            while index < len(targets) and targets[index].floornum < new_floor:
                index += 1
        
        additional_wait_time = 2 * index
        
        return initial_wait_time + additional_wait_time




    

    def setdirection(self, floornum):
        if self.myfloor < floornum:
            self.direction = -1
        elif self.myfloor > floornum:
            self.direction = +1

    def move(self, dt):
        if self.time2 == 0:
            self.time3 = pygame.time.get_ticks()
            self.time2 = 1

        current_ticks = pygame.time.get_ticks()
        diftime = current_ticks - self.time3
        speed1 = (diftime / 500) * self.floorheight
        self.time3 = current_ticks

        if not self.targets:
            return
        if self.direction is None:
            return

        targetfloor = self.targets[0]
        diftime /= 1000
        if self.pusze == 1:
            
            if current_ticks - self.time1 >= 2000:
                self.pusze = 0
        else:
            self.rect.y += self.direction * speed1
           

        for target in self.targets:#עובד רק בגיקסה הפשוטה
            
            target.timewait -= diftime
        
        adjusted_y = abs(self.rect.y - 1000)  
        self.myfloor = math.floor(adjusted_y / self.floorheight)

        if self.direction < 0:
            self.myfloor -= 1

        if self.myfloor == targetfloor.floornum:
            self.dingsound.play()
            self.time2 = 0
            self.time1 = current_ticks
            self.pusze = 1
            targetfloor.timewait = 0
            self.targets.pop(0)
            if not self.targets:
                self.direction = None
                self.moving = False



    def ismovingindirection(self, floornum, direction):
        
        if self.direction is None:
            return True
        return (self.direction == 1 and self.myfloor > floornum) or (self.direction == -1 and self.myfloor < floornum)
    if a88==0:
        def ismovingindirection(self, floornum,a):
            if self.direction is None:
                return True

        
            if self.direction == 1:
            
                return floornum < self.targets[-1].floornum

            
            elif self.direction == -1:
            
                return floornum > self.targets[-1].floornum

    

    


    def draw(self):
        self.screen.blit(self.image, self.rect)

        font = pygame.font.Font(None, 36)  

        text_surface = font.render(str(self.rect.y), True, (0, 0, 0))  
        text_rect = text_surface.get_rect(center=(self.rect.centerx, self.rect.y + 20))  

        
            

class floor:
    def __init__(self, screen, posx, posy, height, width, floornum, building):
        self.screen = screen
        self.posx = posx
        self.posy = posy
        self.width = width
        self.height = height
        self.floornum = floornum
        self.building = building
        self.timewait=0
        
        brickimage = pygame.image.load(imgfloor).convert_alpha()
        self.bricktexture = pygame.transform.scale(brickimage, (width, height))
        self.rect = self.bricktexture.get_rect(topleft=(posx, posy))
        self.buttonrect = pygame.Rect(posx + width - 80, posy + 10, 50, 30)
        
        
    def draw(self):
        
        self.screen.blit(self.bricktexture, self.rect)
        
        
        pygame.draw.ellipse(self.screen, (128, 128, 128), self.buttonrect)

        
        font = pygame.font.Font(None, 36)  
        button_text = font.render(str(self.floornum), True, (255, 255, 255))  
        button_text_rect = button_text.get_rect(center=self.buttonrect.center)
        self.screen.blit(button_text, button_text_rect)
        new_rect = pygame.Rect(self.posx + self.width-30, self.posy + 30, 10, 20)
        transparent_surface = pygame.Surface((new_rect.width, new_rect.height), pygame.SRCALPHA)
        transparent_surface.fill((0, 0, 0, 128))
        self.screen.blit(transparent_surface, new_rect.topleft)
        a888=(self.timewait)*10
        a888=int(a888)
        a888=a888/10
        number_text = font.render(str((a888)), True, (0, 250, 0))
        number_text_rect = number_text.get_rect(center=new_rect.center)
        self.screen.blit(number_text, number_text_rect)
    def checkclick(self, position):
        if self.buttonrect.collidepoint(position):
            self.callelevator()

    def callelevator(self):
        
        self.building.update(self)



def run_game():
    screen_width, screen_height, floor_height, floor_width, num_elevators,numfloors=1000, 1800, 70, 200, 5,17
    pygame.init()
    screen = pygame.display.set_mode((1800, 1000))
    pygame.display.set_caption('Building Simulation')
    clock = pygame.time.Clock()
    print(screen_width/numfloors)
    building_instance = building(screen, screen_width/numfloors, floor_width, num_elevators)
    a=1
    running = True
    while running:
        if a==1:
         dt = clock.tick(60)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                print("Mouse Clicked at:", event.pos)  
                if event.button == 1:  
                    for floor in building_instance.floors:
                        floor.checkclick(event.pos)

        screen.fill((255, 255, 255))  
        building_instance.draw()  
        building_instance.update1(dt)
        pygame.display.flip()  
        clock.tick(60)  

    pygame.quit()


run_game()

58.8235294117647
Mouse Clicked at: (43, 377)
Mouse Clicked at: (43, 495)
Mouse Clicked at: (55, 137)
Mouse Clicked at: (47, 248)
